In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import transformers
from transformers import AutoModel, AutoTokenizer
import matplotlib.pyplot as plt
import sqlite3

In [2]:
train_curve = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
batch_size = 10
epoches = 100
model = "bert-base-chinese"
hidden_size = 768
n_class = 2
maxlen = 8

In [4]:
import pathlib
pathlib.Path().resolve()

PosixPath('/home/tekashi/Documents/Therapy-Chatbot-Deploying-NLP/Training')

In [5]:
cnx = sqlite3.connect('../Scraping/Data/labelTitle.db')
df = pd.read_sql_query("SELECT * FROM labelTitle", cnx)

length = df.shape[0]
tenthLength = length//10

popRank = df.sort_values('poprank',ascending=True)
titles = []
labels = []

for x in range(length):
    titles.append(popRank['title'][x])
    if x <= tenthLength:
        labels.append('1')
    elif x > tenthLength and x <= 2*tenthLength:
        labels.append('2')
    elif x > 2*tenthLength and x <= 3*tenthLength:
        labels.append('3')
    elif x > 3*tenthLength and x <= 4*tenthLength:
        labels.append('4')
    elif x > 4*tenthLength and x <= 5*tenthLength:
        labels.append('5')
    elif x > 5*tenthLength and x <= 6*tenthLength:
        labels.append('6')
    elif x > 6*tenthLength and x <= 7*tenthLength:
        labels.append('7')
    elif x > 7*tenthLength and x <= 8*tenthLength:
        labels.append('8')
    elif x > 8*tenthLength and x <= 9*tenthLength:
        labels.append('9')
    elif x > 9*tenthLength:
        labels.append('10')
for x in range(length):
        print(titles[x], labels[x])

放不下確診離世的家人 1
穿這樣被罵很騷 1
我家人說我在賣肉 1
跟差點被騙去柬埔寨的朋友絕交了 1
可以不要抹黑Bump了嗎 1
怎麼讓我妹相信她真的有超能力 1
我真的很想一棍子打醒我姐 1
！！乾！鞋子倒出一隻蟾蜍 1
鼻子毀了我的臉 1
我鄰居的大哥曾經被緬甸的人蛇集團綁架 1
我的媽媽不承認我是他的孩子 1
機車駕照怎麼考都考不過 1
柬埔寨外流影片看了好不適 1
看見小朋友在公共廁所洗內褲 1
長相比較符合上個時代的審美 1
我是逆媳又怎樣？ 1
柬埔寨無碼血腥影片 1
爸偏心 1
柬埔寨爆料 1
我很想祝福她考上 但我做不到 1
何等渺小卻能被行政院長幫助.... 1
基督徒的我居然為了五萬抄心經 1
爸爸的女友臉皮有夠厚… 1
雖然考過駕照但 1
好想懷孕 1
小時候被學長猥褻 1
要上大學了 要搬宿舍了 好緊張 1
台灣一大堆正妹 1
在另一個世界當位快樂的天使吧🌈艾成……R.I.P…… 1
男人間的幹話⋯嗎？ 1
處女沒怎樣 他媽說謊才是最大的問題好嗎 1
月薪38K 住外面 孝親費一般給多少？ 1
我有一個會家暴的哥哥 1
請大家真的要慎選朋友！！ 1
因為疫苗頭髮掉光 1
林中之光 1
按摩舒壓 1
淺談+9都把得到正妹 1
我表姐喜歡她堂弟 1
今天我生日 1
學開車跟騎機車關係很大嗎？ 1
分手後的成長 1
沒被好好對待過的小孩，該怎麼愛人 1
第一次見面直接上車？ 1
側臉很醜。 1
該怎麼治療恐男 1
曾沛慈《今天陽光就是特別特別耀眼特別和諧》超好聽 1
外籍生被逼著回家 不知能怎麼辦 1
大家還有跟大學以前的朋友聯絡嗎？ 1
深夜看了這篇文，突然讓我想起我的媽媽 1
讓人窒息的家庭 1
台妹猴子行徑 1
艾成好可憐啊！ 1
一個人看演唱會會不會很怪 1
每個月給孝親費的壓力 1
高中學妹自殺 1
旋轉拍賣詐騙/無法下單 1
8/16搭自強117到台南，不小心睡在一位男生的肩膀上，真的是很不好意思，本來想要好好跟他說聲抱歉，但同時在台南站下車，雨下得超級大，後來一眨眼人就不見蹤影了，只想要說聲謝謝你。 1
朋友拉屎拉到一半 1
拔個智齒差點毀容，該求償嗎？ 1
1925的接線生問我「想不想自殺」？ 1
不敢唱歌（有音檔 1
好想確診憂鬱症 1
老闆詐騙😡 1
霸凌的人到底是什麼心態 1
生日快樂 1
關於婆婆 我壓力很大 1
生活累

In [6]:
class MyDataset(Data.Dataset):
    def __init__(self, titles, labels=None, with_labels=True,):
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.with_labels = with_labels
        self.titles = titles
        self.labels = labels
    def __len__(self):
        return len(titles)

    def __getitem__(self, index):
        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent = self.titles[index]

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent,
                        padding='max_length',  # Pad to max_length
                        truncation=True,       # Truncate to max_length
                        max_length=maxlen,  
                        return_tensors='pt')  # Return torch.Tensor objects

        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.labels[index]
            return token_ids, attn_masks, token_type_ids, label
        else:
            return token_ids, attn_masks, token_type_ids

In [7]:
train = Data.DataLoader(dataset=MyDataset(titles, labels), batch_size=batch_size, shuffle=True, num_workers=1)

In [8]:
class BertClassify(nn.Module):
    def __init__(self):
        super(BertClassify, self).__init__()
        self.bert = AutoModel.from_pretrained(model, output_hidden_states=True, return_dict=True)
#       ERROR :  `__init__() got an unexpected keyword argument 'return_dict'`
#         self.bert = AutoModel.from_pretrained(model, output_hidden_states=True)
        self.linear = nn.Linear(hidden_size, n_class)
        self.dropout = nn.Dropout(0.5)

    def forward(self, X):
        input_ids, attention_mask, token_type_ids = X[0], X[1], X[2]
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) # 返回一个output字典
        # 用最后一层cls向量做分类
        # outputs.pooler_output: [bs, hidden_size]
        logits = self.linear(self.dropout(outputs.pooler_output))

In [9]:
bc = BertClassify().to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
optimizer = optim.Adam(bc.parameters(), lr=1e-3, weight_decay=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [11]:
# train
sum_loss = 0
total_step = len(train)
for epoch in range(epoches):
    for i, batch in enumerate(train):
        optimizer.zero_grad()
        # for p in batch:
        #     print(p)
        batch = tuple(p.to(device) for p in batch)
#         Above ERROR : 'tuple' object has no attribute 'to'

#         for p in batch:
#             torch.tensor(p).to(device)
#          Above ERROR : `too many dimensions 'str'`

        pred = bc([batch[0], batch[1], batch[2]])
        loss = loss_fn(pred, batch[3])
        sum_loss += loss.item()

        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print('[{}|{}] step:{}/{} loss:{:.4f}'.format(epoch+1, epoches, i+1, total_step, loss.item()))
    train_curve.append(sum_loss)
    sum_loss = 0

AttributeError: 'tuple' object has no attribute 'to'

In [ ]:
print(transformers.__version__)